# **Trabalho 2 - Processamento de Linguagem Natural**

## Giovana Piorino Vieira de Carvalho - Matrícula: 2022035989

###Você pode rodar uma cópia do código no colab: https://colab.research.google.com/drive/11LOCFq8A8sflhRLUfHzE6bazgEY8cazd?usp=sharing, após adicionar o corpus MacMorpho na pasta. Também é possível acessar pelo github: https://github.com/Giovanapvc/NLP-TP2

## Explicação do modelo utilizado

No código abaixo, um treinamento é realizado a partir do modelo base pablocosta/bertabaporu-base-uncased(disponível em https://huggingface.co/pablocosta/bertabaporu-base-uncased), um modelo BERT treinado a partir de uma base de dados de tweets em português brasileiro.

Assim, o modelo foi treinado para o corpus MacMorpho(também em português brasileiro), de forma a executar a tarefa de POS Tagging, a partir dos arquivos de treino e validação disponibilizados(macmorpho-train e macmorpho-dev). Em seguida, a partir do conjunto de teste(sinalizado pelo arquivo macmorpho-test.txt), o modelo, após o seu treinamento, foi testado de forma a avaliar sua performance. As principais métricas se encontram no classification report realizado.

In [2]:
# A função abaixo carrega os arquivos do corpus MacMorpho e organiza as sentenças e as categorias de POS Tagging utilizadas em treino, teste e validação

def load_corpus(filepath):
    sentences, tags = [], []

    with open(filepath, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:
                words, pos_tags = [], []
                for pair in line.split():
                    if "_" in pair:
                        word, pos = pair.rsplit("_", 1)
                        words.append(word)
                        pos_tags.append(pos)
                    else:
                        print(f"Formato inesperado na linha: {pair}")
                sentences.append(words)
                tags.append(pos_tags)

    return sentences, tags

train_file = "macmorpho-train.txt"
val_file = "macmorpho-dev.txt"
test_file = "macmorpho-test.txt"

train_sentences, train_tags = load_corpus(train_file)
val_sentences, val_tags = load_corpus(val_file)
test_sentences, test_tags = load_corpus(test_file)


In [ ]:
# Construção do mapeamento de tags para índices
unique_tags = set(tag for tags in train_tags for tag in tags)
tag2idx = {tag: idx for idx, tag in enumerate(unique_tags)}
idx2tag = {idx: tag for tag, idx in tag2idx.items()}


In [ ]:
# A função tokeniza sentenças e alinha rótulos com os tokens gerados, garantindo que etiquetas correspondam às palavras originais

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("pablocosta/bertabaporu-base-uncased")

def tokenize_and_align_labels(sentences, tags):
    tokenized_inputs = tokenizer(sentences, truncation=True, is_split_into_words=True, padding=True)
    aligned_labels = []
    for i, label in enumerate(tags):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(tag2idx[label[word_idx]])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        aligned_labels.append(label_ids)
    tokenized_inputs["labels"] = aligned_labels
    return tokenized_inputs

train_encodings = tokenize_and_align_labels(train_sentences, train_tags)
val_encodings = tokenize_and_align_labels(val_sentences, val_tags)
test_encodings = tokenize_and_align_labels(test_sentences, test_tags)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/504k [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
# A classe define um dataset personalizado para manipular os dados tokenizados, facilitando seu uso com
# DataLoaders do PyTorch em tarefas de treinamento e avaliação de modelos.

import torch

class MacMorphoDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


train_dataset = MacMorphoDataset(train_encodings)
val_dataset = MacMorphoDataset(val_encodings)
test_dataset = MacMorphoDataset(test_encodings)


In [ ]:
# Código adicional para configurar o Accelerate

from accelerate import Accelerator, DataLoaderConfiguration

dataloader_config = DataLoaderConfiguration(
    dispatch_batches=False,
    split_batches=True
)

accelerator = Accelerator(dataloader_config=dataloader_config)

##Treinamento do modelo

O código abaixo sumariza os parâmetros utilizados para o treinamento do modelo com ênfase na tarefa de POS Tagging, a partir do corpus MacMorpho, tokenizado anteriormente.

Os parâmetros de treinamento foram definidos usando a classe TrainingArguments, incluindo uma taxa de aprendizado de 5e-5, tamanho de lote de 16 por dispositivo para treinamento e avaliação, três épocas de treinamento, e um peso de regularização (weight decay) de 0.01 para evitar overfitting. O modelo foi configurado para ser avaliado e salvo ao final de cada época, com os logs armazenados em um diretório editável. O treinamento foi gerenciado pela classe Trainer, que recebeu o modelo, os argumentos, os conjuntos de dados de treinamento e validação, e o tokenizador para preparar as entradas. O processo foi iniciado com train(), permitindo a otimização dos pesos do modelo e o salvamento dos checkpoints ao longo do treinamento.

Na primeira época, a perda de treinamento foi de 0.1035 e a de validação foi de 0.0901. Na segunda época, a perda de treinamento diminuiu para 0.0560, enquanto a perda de validação também reduziu ligeiramente para 0.0873. Na terceira época, a perda de treinamento caiu para 0.0291, mas a perda de validação aumentou levemente para 0.0930, sugerindo que o modelo pode estar começando sofrer overfitting, e possíveis ajustes de parâmetros podem ser feitos em outro momento para evitar essa tendência.

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    "pablocosta/bertabaporu-base-uncased",
    num_labels=len(tag2idx)
)

training_args = TrainingArguments(
    output_dir="./",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    save_strategy="epoch",
    report_to=[]
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

trainer.train()

Some weights of BertForTokenClassification were not initialized from the model checkpoint at pablocosta/bertabaporu-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-9-343a8c10cda0>:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,0.103500,0.090088
2,0.056000,0.087330
3,0.029100,0.093043


TrainOutput(global_step=7116, training_loss=0.07619196079365653, metrics={'train_runtime': 4459.1753, 'train_samples_per_second': 25.53, 'train_steps_per_second': 1.596, 'total_flos': 1.2726604379669904e+16, 'train_loss': 0.07619196079365653, 'epoch': 3.0})

##Avaliação da precisão do modelo para a tarefa de POS Tagging

No geral, o modelo apresentou bom desempenho, com uma precisão geral de 0.97 e um F1-score médio ponderado de 0.97. Algumas classes específicas, como "PU" (pontuação), "V" (verbos) e "ART" (artigos), atingiram F1-scores próximos ou iguais a 1.00, indicando que o modelo tem uma alta capacidade de identificar esses rótulos de forma consistente. Uma possível explicação é a singularidade dessas categorias, uma vez que pontuações representam conjuntos de símbolos específicos apenas para o propósito dessa classificação. Artigos tambḿ compõem uma classe mais restrita de palavras, e não apresentam tanta variedade de uso gramatical. Os verbos, por serem estruturas centrais em sentenças e orações, também apresentam padrões de terminações devido às suas conjugações, uma possível explicação para a maior facilidade na rotulação.

Em contrapartida, a classe IN(Interjeições) apresentou o menor F1-Score(0.56), e bem abaixo da média dessa métrica em relação às outras tags. Um possível fator que dificultou a rotulação dessa categoria é a análise de contexto que deve-se realizar em relação a ela: no manual das tags do corpus MacMorpho,as interjeições podem ser caracterizadas como invocações, chamamentos, demonstrações de interesse e surpresa, xingamentos, e palavras que sinalizam estados de emoção em geral. A alta diversidade de representações e a sinalização de "sentimentos" em uma sentença(levando a uma interpretação mais sofisticada) dessa tag pode ser um fator de maior dificuldade de classificação por parte do modelo.

Outras classes, como PREP+PRO-KS(preposição com pronome conectivo subordinativo) e ADV-KS(advérbio conectivo subordinativo), apresentaram F1-scores mais baixos também(em torno de 0.85 a 0.87), possivelmente devido ao menor número de exemplos apresentados, o que pode dificultar o aprendizado do modelo.

O desempenho macro médio, que considera todas as classes igualmente, também foi alto, com um F1-score de 0.93, refletindo a capacidade do modelo de generalizar bem para diferentes rótulos. A discrepância entre as métricas macro e ponderada indica que o modelo lida melhor com classes majoritárias do que com classes menos frequentes. No entanto, as métricas gerais indicam que o modelo BERTabaporu é eficaz e adequado para a tarefa de classificação de tokens, com desempenho robusto na maioria das classes.

In [ ]:
# Avaliação do desempenho do modelo no conjunto de teste, gerando um relatório de classificação

from sklearn.metrics import classification_report
import numpy as np

label_list = list(tag2idx.keys())

predictions, true_labels, _ = trainer.predict(test_dataset)
pred_labels = np.argmax(predictions, axis=2)

# Excluir os rótulos de padding se necessário (dependendo do seu conjunto de dados)
mask = true_labels != -100  # Assumindo que -100 é o valor de padding

true_labels = true_labels[mask]
pred_labels = pred_labels[mask]

report = classification_report(true_labels, pred_labels, target_names=label_list)
print(report)

Epoch,Training Loss,Validation Loss


              precision    recall  f1-score   support

 PREP+PRO-KS       0.88      0.86      0.87        58
      PROADJ       0.97      0.98      0.97      3419
    PREP+ADV       0.97      0.90      0.93        31
         CUR       0.99      1.00      0.99       296
          IN       0.49      0.64      0.56        98
         ADJ       0.95      0.95      0.95      8554
         NUM       0.96      0.96      0.96      2541
        PDEN       0.88      0.91      0.90      1092
          KC       0.97      0.97      0.97      4531
       NPROP       0.94      0.93      0.94     15936
    PREP+ART       0.98      0.98      0.98     10219
          KS       0.92      0.92      0.92      2538
 PREP+PROSUB       0.89      0.85      0.87       156
         ART       0.98      0.99      0.99     12580
 PREP+PROADJ       1.00      0.99      1.00       309
           V       0.99      0.99      0.99     19711
         PCP       0.97      0.96      0.96      3640
PREP+PROPESS       1.00    

In [ ]:
# Salvando o modelo

save_directory = "./trained_model"
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

Modelo e tokenizador salvos no diretório: ./trained_model


In [ ]:
from transformers import AutoModelForTokenClassification, AutoTokenizer

# Carregar o modelo e o tokenizador salvos
model = AutoModelForTokenClassification.from_pretrained("./trained_model")
tokenizer = AutoTokenizer.from_pretrained("./trained_model")